In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection
nltk.download('punkt')

from transformers import BertTokenizer

[nltk_data] Downloading package punkt to /home/moonstar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-06-25 10:40:43.925235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 10:40:45.027978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/home/moonstar/.mujoco/mujoco210/bin:/usr/lib/nvidia
2023-06-25 10:40:45.028165: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot

In [2]:
df_train = pd.read_csv('open/train.csv', delimiter=',', encoding = "utf8")
print(len(df_train))

# NaN값 삭제
df_train.dropna(inplace=True)
print(len(df_train))

print(df_train['facts'][0])
df_train.head()

2478
2478
On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. 
Thompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court held that the First Amendment protects uninhibited, robust debate, 

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [3]:
sentences = []
first_people = []
second_people = []
winners = []


for j in range(len(df_train)):

    first_party = df_train['first_party'][j]
    second_party = df_train['second_party'][j]
    # print(first_party)
    # print(second_party)

    first_party_tokens = nltk.tokenize.word_tokenize(first_party)
    second_party_tokens = nltk.tokenize.word_tokenize(second_party)
    # print(first_party_tokens)
    # print(second_party_tokens)


    fact = df_train['facts'][j]
    fact_tokens = nltk.tokenize.word_tokenize(fact)

    first = 'tony'
    second = 'jack'

    first_people.append(first)
    # first_people.append(second)
    second_people.append(second)
    # second_people.append(first)

    win = df_train['first_party_winner'][j]
    winners.append(win)
    # winners.append((win*-1)+1)

    name_tmp = []
    new = []
    new2 = []

    new_sentence1 = ''
    new_sentence2 = ''

    for i in range(len(fact_tokens)):
        token = fact_tokens[i]
        token2 = fact_tokens[i]
        plus = True

        if token in first_party_tokens:
            if token == first_party_tokens[-1]:
                token = first 
                token2 = second

            else:
                token = ''
                plus = False 

        if token in second_party_tokens:
            if token == second_party_tokens[-1]:
                token = second 
                token2 = first 

            else:
                token = ''
                plus = False

        if plus == True:
            new.append(token)
            # new2.append(token2)

            new_sentence1 += token + ' '
            # new_sentence2 += token2 + ' '


    # print(new)
    # print(new2)

    # print(new_sentence1)
    # print(new_sentence2)

    sentences.append(new_sentence1)
    # sentences.append(new_sentence2)


In [4]:
# sentences = []
# first_people = []
# second_people = []
# winners = []

# print(sentences[:5])
print(first_people[:6])
print(second_people[:6])
print(winners[:20])

['tony', 'tony', 'tony', 'tony', 'tony', 'tony']
['jack', 'jack', 'jack', 'jack', 'jack', 'jack']
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]


In [5]:
first_party_data_frame = pd.DataFrame(first_people, columns=['first_party'])
second_party_data_frame = pd.DataFrame(second_people, columns=['second_party'])
facts_data_frame = pd.DataFrame(sentences, columns=['facts'])
winners_data_frame = pd.DataFrame(winners, columns=['first_party_winner'])

train_data_frame = pd.concat([first_party_data_frame, second_party_data_frame, facts_data_frame, winners_data_frame],axis=1,join='inner')
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
2473,tony,jack,Congress amended the Clean Air Act through the...,1
2474,tony,jack,Inc. an investment fund purchased approximatel...,1
2475,tony,jack,"In 1992 , the District Court sentenced Manuel ...",0
2476,tony,jack,"On March 8 , 1996 , Enrico jack , a lawful per...",0


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
2473,tony,jack,Congress amended the Clean Air Act through the...,1
2474,tony,jack,Inc. an investment fund purchased approximatel...,1
2475,tony,jack,"In 1992 , the District Court sentenced Manuel ...",0
2476,tony,jack,"On March 8 , 1996 , Enrico jack , a lawful per...",0


In [16]:
from datasets import load_dataset

# dataset = load_dataset("yelp_review_full")
# print(dataset["train"][100])

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



: 

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Change num_labels as per your classification task


# print(train_data_frame['facts'])

sentences = []
labels = []
epochs = 1

for i in range(len(train_data_frame)):
# for i in range(1):
    sentence = train_data_frame['facts'][i]
    sentences.append(sentence)

    label = train_data_frame['first_party_winner'][i]
    labels.append(label)

# print(sentences[0])
# print(sentences[1])

print(len(sentences))
print(len(labels))

# sentences = ['This is a positive sentence.', 'This is a negative sentence.']
# labels = [1, 0]

# # Example text classification data
# texts = ['This is a positive sentence.', 'This is a negative sentence.']
# labels = [1, 0]

# Split the data into train and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)
# print(train_labels)
# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
# print(train_encodings)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Create PyTorch datasets
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextClassificationDataset(train_encodings, train_labels)
# print(train_dataset)
val_dataset = TextClassificationDataset(val_encodings, val_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(epochs):  # Adjust the number of epochs as needed

    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    preds = []
    true_labels = []

    for batch in tqdm(val_loader):
        inputs = {key: val.to(device) for key, val in batch.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(inputs['labels'].tolist())

    val_accuracy = accuracy_score(true_labels, preds)

    print(f"Epoch: {epoch+1}")
    print(f"Training Loss: {avg_train_loss}")
    print(f"Validation Accuracy: {val_accuracy}")
    print("---------------------------")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

2478
2478


/home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
100%|██████████| 248/248 [00:11<00:00, 22.06it/s]

Epoch: 1
Training Loss: 0.8173600978074232
Validation Accuracy: 0.3387096774193548
---------------------------


In [8]:
xfeatures = train_data_frame['facts']
ylabel = train_data_frame['first_party_winner']

In [9]:
print(len(xfeatures))

xfeatures += xfeatures



4956


In [10]:
X_train, X_test, y_train, y_test = train_test_split(xfeatures,ylabel, test_size=0.25)

X_train = xfeatures
y_train = ylabel




# k = 1000
# X_train = xfeatures[:k]
# y_train = ylabel[:k]

# X_test = xfeatures[k:]
# y_test = ylabel[k:]

print(len(X_train))
print(len(X_test))

4956
1239


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        lstm_out, _ = self.lstm(input)
        output = self.fc(lstm_out[-1])
        return output

# Prepare the training data
data = [[i for i in range(100)]]
data = torch.tensor(data, dtype=torch.float)
target = [[i for i in range(1, 101)]]
target = torch.tensor(target, dtype=torch.float)

# Reshape the data to match the expected input shape of the LSTM model
data = data.view(1, 1, -1)

# Define the model, loss function, and optimizer
input_size = 1
hidden_size = 10
output_size = 1

model = LSTMModel(input_size, hidden_size, output_size)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the LSTM model
num_epochs = 100

for epoch in range(num_epochs):
    model.zero_grad()

    output = model(data)
    loss = loss_fn(output, target)

    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [69]:
pipe = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(solver='liblinear'))])
pipe.set_params(lr__C=0.0001)
pipe.named_steps['lr'].set_params(max_iter=100000)
# # Access the LogisticRegression object within the pipeline
# lr_model = pipe.named_steps['lr']

# # Set the number of epochs
# lr_model.max_iter = 10000

LogisticRegression(C=0.0001, max_iter=100000, solver='liblinear')

In [70]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr',
                 LogisticRegression(C=0.0001, max_iter=100000,
                                    solver='liblinear'))])

In [71]:
pipe.score(X_test,y_test)

# y_pred = pipe.predict(X_test)
# print(y_pred)

0.5076674737691687

In [60]:
"""
Test data
"""


df_test = pd.read_csv('open/test.csv', delimiter=',', encoding = "utf8")
print(len(df_test))

# NaN값 삭제
df_test.dropna(inplace=True)
print(len(df_test))

print(df_test['facts'][0])
df_test.head()

1240
1240
The 1984 Bail Reform Act allowed the federal courts to detain an arrestee prior to trial if the government could prove that the individual was potentially dangerous to other people in the community. Prosecutors alleged that Salerno and another person in this case were prominent figures in the La Cosa Nostra crime family.



,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [61]:
sentences = []
first_people = []
second_people = []


for j in range(len(df_test)):

    first_party = df_test['first_party'][j]
    second_party = df_test['second_party'][j]
    # print(first_party)
    # print(second_party)

    first_party_tokens = nltk.tokenize.word_tokenize(first_party)
    second_party_tokens = nltk.tokenize.word_tokenize(second_party)
    # print(first_party_tokens)
    # print(second_party_tokens)


    fact = df_test['facts'][j]
    fact_tokens = nltk.tokenize.word_tokenize(fact)

    first = 'tony'
    second = 'jack'

    first_people.append(first)
    # first_people.append(second)
    second_people.append(second)
    # second_people.append(first)

    name_tmp = []
    new = []
    new2 = []

    new_sentence1 = ''
    new_sentence2 = ''

    for i in range(len(fact_tokens)):
        token = fact_tokens[i]
        token2 = fact_tokens[i]
        plus = True

        if token in first_party_tokens:
            if token == first_party_tokens[-1]:
                token = first 
                token2 = second

            else:
                token = ''
                plus = False 

        if token in second_party_tokens:
            if token == second_party_tokens[-1]:
                token = second 
                token2 = first 

            else:
                token = ''
                plus = False

        if plus == True:
            new.append(token)
            # new2.append(token2)

            new_sentence1 += token + ' '
            # new_sentence2 += token2 + ' '


    # print(new)
    # print(new2)

    # print(new_sentence1)
    # print(new_sentence2)

    sentences.append(new_sentence1)
    # sentences.append(new_sentence2)


In [62]:
print(len(sentences))
print(len(first_people))
print(len(second_people))

first_party_data_frame = pd.DataFrame(first_people, columns=['first_party'])
second_party_data_frame = pd.DataFrame(second_people, columns=['second_party'])
facts_data_frame = pd.DataFrame(sentences, columns=['facts'])

test_data_frame = pd.concat([first_party_data_frame, second_party_data_frame, facts_data_frame],axis=1,join='inner')
test_data_frame

1240
1240
1240


,first_party,second_party,facts
0,tony,jack,The 1984 Bail Reform Act allowed the federal c...
1,tony,jack,Inc. was a defendant in a class action lawsuit...
2,tony,jack,In 2002 and 2003 broadcast the Billboard Music...
3,tony,jack,During his trial for armed robbery of a federa...
4,tony,jack,"In 1993 , a magistrate judge issued a warrant ..."
...,...,...,...
1235,tony,jack,According to Executive Order No tony 12807 sig...
1236,tony,jack,Section 109 ( a ) of the Clean Air Act ( CAA )...
1237,tony,jack,tony created a plan for utilizing $ 2.6 millio...
1238,tony,jack,"In 1972 , the North Carolina Board of Agricult..."


In [101]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Change num_labels as per your classification task

sentences = []
epochs = 100

for i in range(len(test_data_frame)):

    sentence = test_data_frame['facts'][i]
    sentences.append(sentence)

print(len(sentences))

test_texts = sentences
test_labels = [0]*len(test_texts)

# print(test_texts)
print(len(test_texts))
# print(test_labels)
print(len(test_labels))

# Tokenize the texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Create PyTorch datasets
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TextClassificationDataset(test_encodings, test_labels)

# Create data loaders
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

model.eval()
preds = []
true_labels = []

for batch in tqdm(test_loader):
    inputs = {key: val.to(device) for key, val in batch.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    preds.extend(torch.argmax(logits, dim=1).tolist())
    true_labels.extend(inputs['labels'].tolist())

y_pred = preds

print('========= y_pred ========')
print(preds)
print(y_pred)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_894567/2227742880.py:10 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_894567/2227742880.py'                       │
│                                                                                                  │
│ /home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/modeling_utils.py │
│ :1979 in from_pretrained                                                                         │
│                                                                                                  │
│   1976 │   │   │   │   subfolder=subfolder,                                                      │
│   1977 │   │   │   │   _from_auto=from_auto_class,                                               │
│   1978 │   │   │   │   _from_pipeline=from_pipeline,                                             │
│ ❱ 1979 │   │   │   │   **kwargs,                                                                 │
│   1980 │   │   │   )                                                                             │
│   1981 │   │   else:                                                                             │
│   1982 │   │   │   model_kwargs = kwargs                                                         │
│                                                                                                  │
│ /home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/configuration_uti │
│ ls.py:532 in from_pretrained                                                                     │
│                                                                                                  │
│   529 │   │   assert config.output_attentions == True                                            │
│   530 │   │   assert unused_kwargs == {"foo": False}                                             │
│   531 │   │   ```"""                                                                             │
│ ❱ 532 │   │   config_dict, kwargs = cls.get_config_dict(pretrained_model_name_or_path, **kwarg   │
│   533 │   │   if "model_type" in config_dict and hasattr(cls, "model_type") and config_dict["m   │
│   534 │   │   │   logger.warning(                                                                │
│   535 │   │   │   │   f"You are using a model of type {config_dict['model_type']} to instantia   │
│                                                                                                  │
│ /home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/configuration_uti │
│ ls.py:559 in get_config_dict                                                                     │
│                                                                                                  │
│   556 │   │   """                                                                                │
│   557 │   │   original_kwargs = copy.deepcopy(kwargs)                                            │
│   558 │   │   # Get config dict associated with the base config file                             │
│ ❱ 559 │   │   config_dict, kwargs = cls._get_config_dict(pretrained_model_name_or_path, **kwar   │
│   560 │   │   if "_commit_hash" in config_dict:                                                  │
│   561 │   │   │   original_kwargs["_commit_hash"] = config_dict["_commit_hash"]                  │
│   562                                                                                            │
│                                                                                                  │
│ /home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/configuration_uti │
│ ls.py:626 in _get_config_dict                              

In [102]:
y_pred = pd.DataFrame(y_pred, columns=['first_party_winner'])
y_pred

,first_party_winner
0,0
1,0
2,0
3,0
4,0
...,...
1235,1
1236,0
1237,1
1238,1


In [103]:
prediction = pd.concat([df_test['ID'],y_pred],axis=1,join='inner')
prediction

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,1
1236,TEST_1236,0
1237,TEST_1237,1
1238,TEST_1238,1


In [104]:
prediction.to_csv('./baseline_submit2.csv', index=False)
print('Done')

Done


In [75]:
submit = pd.read_csv('open/sample_submission.csv')
print(submit[:5])

          ID  first_party_winner
0  TEST_0000                   0
1  TEST_0001                   0
2  TEST_0002                   0
3  TEST_0003                   0
4  TEST_0004                   0
